In [9]:
import pandas as pd
import numpy as np
import string 
import random 
import time

### Generating a CSV file with random data

In [4]:
# setting the number of rows for the CSV file (1M)
N = 1000000

# creating a pandas dataframe (df) with 8 columns and N rows with random integers between 999 and 999999 and with column names from A to H
df = pd.DataFrame(
    np.random.randint(9,999, size=(N, 7)), 
    columns=list('ABCDEFG'))

# creating one column 'H' of float type using the uniform distribution
df['H'] = np.random.rand(N)

# creating two additional columns with random strings
df['I'] = pd.util.testing.rands_array(10, N)
df['J'] = pd.util.testing.rands_array(10, N)

# expect: 1M rows x 10 columns
df.shape

(1000000, 10)

In [5]:
# export the dataframe to csv ~75MB
df.to_csv("test_data_1M.csv")

In [ ]:
start_time = time.time()
do_something()
print("%s seconds" % (time.time() - start_time))